In [1]:
# Importar librerias
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from pandas.core.arrays import categorical # agregar class pandas.Categorical(values, categories=None, ordered=None, dtype=None, fastpath=False, copy=True)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
import re

In [2]:
# Importar df downtime limpiio
downtime = pd.read_csv("/content/downtime.csv")

In [3]:
# Visualiza Dataset
downtime

,EventID,ScheduleID,AssetName,DownStart,DownEnd,Reason,Category,DownStart_dia,DownStart_mes,DownStart_año,Total_Duration,Fault_Number,Category_Number,ShiftNumber,OEE
0,1052248,25579,Bench 100D,2022-01-06,2022-01-06,Falla 49: Lifter MarkerPosn. Superior,Position Sensor Faults,6,1,2022,12,49,3,1,1.0
1,1052250,25579,Bench 100D,2022-01-06,2022-01-06,Falla 58: C-Clip Posn. Extendida,Position Sensor Faults,6,1,2022,2,58,3,1,1.0
2,1052253,25579,Bench 100D,2022-01-06,2022-01-06,Falla189. Paremtro Close lenght Tolerancia Act...,Position Sensor Faults,6,1,2022,1,189,3,1,1.0
3,1052265,25579,Bench 100D,2022-01-06,2022-01-06,Falla 49: Lifter MarkerPosn. Superior,Position Sensor Faults,6,1,2022,1,49,3,1,1.0
4,1052274,25579,Bench 100D,2022-01-06,2022-01-06,Falla 102: Disponible,Other Faults,6,1,2022,8,102,9,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50950,1717115,59167,Bench 100D,2024-05-07,2024-05-07,Falla35. Servo Pin Izquierdo Longitud Fallo,Servo Support Faults,7,5,2024,1,35,0,2,1.0
50951,1717120,59167,Bench 100D,2024-05-07,2024-05-07,Falla35. Servo Pin Izquierdo Longitud Fallo,Servo Support Faults,7,5,2024,1,35,0,2,1.0
50952,1717121,59167,Bench 100D,2024-05-07,2024-05-07,Falla 12. Scanner de Seguridad Interrumpido Du...,Scanner Faults,7,5,2024,2,12,6,2,1.0
50953,1717122,59167,Bench 100D,2024-05-07,2024-05-07,Falla35. Servo Pin Izquierdo Longitud Fallo,Servo Support Faults,7,5,2024,1,35,0,2,1.0


In [4]:
# Importar DowntimeSummary Dataset
DowntimeSummary = pd.read_csv("/content/Ensamble 100D production history DB2 22-23-24.csv")

In [5]:
#Imprime DF
DowntimeSummary

,Department,Work Group,Asset Name,Total Count,Count,Rejects,Total Downtime,Fault Count,MTTR,MTBF,...,Planned Rate,Product,Productivity OEE,Availability OEE,Quality OEE,OEE,Shift Number,Start Date,End Date,ScheduleID
0,100D,Ensamble 100D,Bench 100D,0,0,0,600,0,0,0,...,15.7,0.0,0.0,0.000000,1,0.0,1,1/3/2022,1/3/2022,25576
1,100D,Ensamble 100D,Bench 100D,0,0,0,539,0,0,0,...,0.0,0.0,0.0,0.000000,1,0.0,2,1/3/2022,1/4/2022,31538
2,100D,Ensamble 100D,Bench 100D,0,0,0,600,0,0,0,...,15.7,0.0,0.0,0.000000,1,0.0,1,1/4/2022,1/4/2022,25577
3,100D,Ensamble 100D,Bench 100D,0,0,0,539,0,0,0,...,0.0,0.0,0.0,0.000000,1,0.0,2,1/4/2022,1/5/2022,31473
4,100D,Ensamble 100D,Bench 100D,0,0,0,600,0,0,0,...,15.7,0.0,0.0,0.000000,1,0.0,1,1/5/2022,1/5/2022,25578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,100D,Ensamble 100D,Bench 100D,79,79,0,1,17,0,30,...,20.0,0.0,0.0,0.998039,1,0.0,1,5/4/2024,5/4/2024,59182
953,100D,Ensamble 100D,Bench 100D,111,111,0,42,93,0,6,...,20.0,0.0,0.0,0.930000,1,0.0,1,5/6/2024,5/6/2024,59164
954,100D,Ensamble 100D,Bench 100D,98,98,0,42,79,0,6,...,20.0,0.0,0.0,0.922222,1,0.0,2,5/6/2024,5/7/2024,59165
955,100D,Ensamble 100D,Bench 100D,153,153,0,63,115,0,5,...,20.0,0.0,0.0,0.895000,1,0.0,1,5/7/2024,5/7/2024,59166


In [6]:
# Eliminar columnas que no estan relacionadas al analisis
DowntimeSummary.drop(["Department", "Work Group",	"Total Count","Count","Rejects","Fault Count","MTTR","MTBF","Total Downtime","Total Uptime","Total Breaks","Total Working Time","Production Rate","Product","Productivity OEE","Availability OEE","Quality OEE","OEE","Asset Name","End Date","Start Date","Shift Number"], axis = 1, inplace = True)

In [7]:
#Imprime DF
DowntimeSummary

,Total Time,Production Goal,Planned Rate,ScheduleID
0,600,157,15.7,25576
1,539,0,0.0,31538
2,600,157,15.7,25577
3,539,0,0.0,31473
4,600,157,15.7,25578
...,...,...,...,...
952,510,0,20.0,59182
953,600,0,20.0,59164
954,540,0,20.0,59165
955,600,0,20.0,59166


In [8]:
# Utiliza la función merge para combinar los dataframes basados en la columna ScheduleID
TablaMaestraDowntime = pd.merge(downtime, DowntimeSummary, on='ScheduleID', how='left')

# El parámetro 'how' indica cómo realizar la unión. En este caso, estamos usando 'left',
# lo que significa que conservaremos todas las filas del dataframe dowtime,
# y solo añadiremos las columnas de downtime2 que coincidan con ScheduleID.

# Visualizamos el nuevo dataframe
TablaMaestraDowntime

,EventID,ScheduleID,AssetName,DownStart,DownEnd,Reason,Category,DownStart_dia,DownStart_mes,DownStart_año,Total_Duration,Fault_Number,Category_Number,ShiftNumber,OEE,Total Time,Production Goal,Planned Rate
0,1052248,25579,Bench 100D,2022-01-06,2022-01-06,Falla 49: Lifter MarkerPosn. Superior,Position Sensor Faults,6,1,2022,12,49,3,1,1.0,600,157,15.7
1,1052250,25579,Bench 100D,2022-01-06,2022-01-06,Falla 58: C-Clip Posn. Extendida,Position Sensor Faults,6,1,2022,2,58,3,1,1.0,600,157,15.7
2,1052253,25579,Bench 100D,2022-01-06,2022-01-06,Falla189. Paremtro Close lenght Tolerancia Act...,Position Sensor Faults,6,1,2022,1,189,3,1,1.0,600,157,15.7
3,1052265,25579,Bench 100D,2022-01-06,2022-01-06,Falla 49: Lifter MarkerPosn. Superior,Position Sensor Faults,6,1,2022,1,49,3,1,1.0,600,157,15.7
4,1052274,25579,Bench 100D,2022-01-06,2022-01-06,Falla 102: Disponible,Other Faults,6,1,2022,8,102,9,1,1.0,600,157,15.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50950,1717115,59167,Bench 100D,2024-05-07,2024-05-07,Falla35. Servo Pin Izquierdo Longitud Fallo,Servo Support Faults,7,5,2024,1,35,0,2,1.0,336,0,20.0
50951,1717120,59167,Bench 100D,2024-05-07,2024-05-07,Falla35. Servo Pin Izquierdo Longitud Fallo,Servo Support Faults,7,5,2024,1,35,0,2,1.0,336,0,20.0
50952,1717121,59167,Bench 100D,2024-05-07,2024-05-07,Falla 12. Scanner de Seguridad Interrumpido Du...,Scanner Faults,7,5,2024,2,12,6,2,1.0,336,0,20.0
50953,1717122,59167,Bench 100D,2024-05-07,2024-05-07,Falla35. Servo Pin Izquierdo Longitud Fallo,Servo Support Faults,7,5,2024,1,35,0,2,1.0,336,0,20.0


In [9]:
# Contar los valores nulos por columna
null_counts = TablaMaestraDowntime.isnull().sum()

# Ordenar las columnas por el total de nulos de mayor a menor
sorted_null_counts = null_counts.sort_values(ascending=False)

# Mostrar el listado de columnas ordenadas por el total de nulos de mayor a menor
print("Listado de columnas ordenadas por el total de nulos de mayor a menor:")
print(sorted_null_counts)

Listado de columnas ordenadas por el total de nulos de mayor a menor:
EventID            0
ScheduleID         0
Production Goal    0
Total Time         0
OEE                0
ShiftNumber        0
Category_Number    0
Fault_Number       0
Total_Duration     0
DownStart_año      0
DownStart_mes      0
DownStart_dia      0
Category           0
Reason             0
DownEnd            0
DownStart          0
AssetName          0
Planned Rate       0
dtype: int64


In [10]:
# Convertir la columna 'DownStart' al formato de fecha y hora
TablaMaestraDowntime['DownStart'] = pd.to_datetime(TablaMaestraDowntime['DownStart'])

# Crear una nueva columna que contenga el número del día de la semana
TablaMaestraDowntime['DayOfWeek'] = TablaMaestraDowntime['DownStart'].dt.dayofweek

# Mostrar el dataframe con la nueva columna
(TablaMaestraDowntime)

,EventID,ScheduleID,AssetName,DownStart,DownEnd,Reason,Category,DownStart_dia,DownStart_mes,DownStart_año,Total_Duration,Fault_Number,Category_Number,ShiftNumber,OEE,Total Time,Production Goal,Planned Rate,DayOfWeek
0,1052248,25579,Bench 100D,2022-01-06,2022-01-06,Falla 49: Lifter MarkerPosn. Superior,Position Sensor Faults,6,1,2022,12,49,3,1,1.0,600,157,15.7,3
1,1052250,25579,Bench 100D,2022-01-06,2022-01-06,Falla 58: C-Clip Posn. Extendida,Position Sensor Faults,6,1,2022,2,58,3,1,1.0,600,157,15.7,3
2,1052253,25579,Bench 100D,2022-01-06,2022-01-06,Falla189. Paremtro Close lenght Tolerancia Act...,Position Sensor Faults,6,1,2022,1,189,3,1,1.0,600,157,15.7,3
3,1052265,25579,Bench 100D,2022-01-06,2022-01-06,Falla 49: Lifter MarkerPosn. Superior,Position Sensor Faults,6,1,2022,1,49,3,1,1.0,600,157,15.7,3
4,1052274,25579,Bench 100D,2022-01-06,2022-01-06,Falla 102: Disponible,Other Faults,6,1,2022,8,102,9,1,1.0,600,157,15.7,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50950,1717115,59167,Bench 100D,2024-05-07,2024-05-07,Falla35. Servo Pin Izquierdo Longitud Fallo,Servo Support Faults,7,5,2024,1,35,0,2,1.0,336,0,20.0,1
50951,1717120,59167,Bench 100D,2024-05-07,2024-05-07,Falla35. Servo Pin Izquierdo Longitud Fallo,Servo Support Faults,7,5,2024,1,35,0,2,1.0,336,0,20.0,1
50952,1717121,59167,Bench 100D,2024-05-07,2024-05-07,Falla 12. Scanner de Seguridad Interrumpido Du...,Scanner Faults,7,5,2024,2,12,6,2,1.0,336,0,20.0,1
50953,1717122,59167,Bench 100D,2024-05-07,2024-05-07,Falla35. Servo Pin Izquierdo Longitud Fallo,Servo Support Faults,7,5,2024,1,35,0,2,1.0,336,0,20.0,1


In [11]:
# Contar la frecuencia de cada valor en la columna 'DayOfWeek'
frequency_counts = (TablaMaestraDowntime)['DayOfWeek'].value_counts()

# Mostrar la frecuencia de cada valor
print("Frecuencia de cada valor en la columna 'DayOfWeek':")
print(frequency_counts)

Frecuencia de cada valor en la columna 'DayOfWeek':
DayOfWeek
1    11064
2    10829
3    10816
0     9347
4     7832
5      975
6       92
Name: count, dtype: int64


In [12]:
# Evaluar los datatypes de cada columna del DF TablaMaestraDowntime

TablaMaestraDowntime.dtypes


EventID                     int64
ScheduleID                  int64
AssetName                  object
DownStart          datetime64[ns]
DownEnd                    object
Reason                     object
Category                   object
DownStart_dia               int64
DownStart_mes               int64
DownStart_año               int64
Total_Duration              int64
Fault_Number                int64
Category_Number             int64
ShiftNumber                 int64
OEE                       float64
Total Time                  int64
Production Goal             int64
Planned Rate              float64
DayOfWeek                   int32
dtype: object

In [13]:
# Renombrar las columnas Total Time a Total_Time, Production Goal a Production_Goal, Planned Rate a Planned_Rate

TablaMaestraDowntime.rename(columns={'Total Time': 'Total_Time', 'Production Goal': 'Production_Goal', 'Planned Rate': 'Planned_Rate'}, inplace=True)


In [14]:
# Visualizamos el dataframe TablaMaestraDowntime
TablaMaestraDowntime

,EventID,ScheduleID,AssetName,DownStart,DownEnd,Reason,Category,DownStart_dia,DownStart_mes,DownStart_año,Total_Duration,Fault_Number,Category_Number,ShiftNumber,OEE,Total_Time,Production_Goal,Planned_Rate,DayOfWeek
0,1052248,25579,Bench 100D,2022-01-06,2022-01-06,Falla 49: Lifter MarkerPosn. Superior,Position Sensor Faults,6,1,2022,12,49,3,1,1.0,600,157,15.7,3
1,1052250,25579,Bench 100D,2022-01-06,2022-01-06,Falla 58: C-Clip Posn. Extendida,Position Sensor Faults,6,1,2022,2,58,3,1,1.0,600,157,15.7,3
2,1052253,25579,Bench 100D,2022-01-06,2022-01-06,Falla189. Paremtro Close lenght Tolerancia Act...,Position Sensor Faults,6,1,2022,1,189,3,1,1.0,600,157,15.7,3
3,1052265,25579,Bench 100D,2022-01-06,2022-01-06,Falla 49: Lifter MarkerPosn. Superior,Position Sensor Faults,6,1,2022,1,49,3,1,1.0,600,157,15.7,3
4,1052274,25579,Bench 100D,2022-01-06,2022-01-06,Falla 102: Disponible,Other Faults,6,1,2022,8,102,9,1,1.0,600,157,15.7,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50950,1717115,59167,Bench 100D,2024-05-07,2024-05-07,Falla35. Servo Pin Izquierdo Longitud Fallo,Servo Support Faults,7,5,2024,1,35,0,2,1.0,336,0,20.0,1
50951,1717120,59167,Bench 100D,2024-05-07,2024-05-07,Falla35. Servo Pin Izquierdo Longitud Fallo,Servo Support Faults,7,5,2024,1,35,0,2,1.0,336,0,20.0,1
50952,1717121,59167,Bench 100D,2024-05-07,2024-05-07,Falla 12. Scanner de Seguridad Interrumpido Du...,Scanner Faults,7,5,2024,2,12,6,2,1.0,336,0,20.0,1
50953,1717122,59167,Bench 100D,2024-05-07,2024-05-07,Falla35. Servo Pin Izquierdo Longitud Fallo,Servo Support Faults,7,5,2024,1,35,0,2,1.0,336,0,20.0,1


In [15]:
# Guarda el data set depurado
TablaMaestraDowntime.to_csv("/content/TablaMaestraDowntime.csv", index= True, header = True)